In [2]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="quick-starts-ws-162706", resource_group='aml-quickstarts-162706', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55')

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-162835
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-162835


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name="udacityprojclust"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('cpu cluster already exist. Using it.')
except ComputeTargetException:

    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

cpu cluster already exist. Using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Install some dependencies for AutoML to run.

import sys

! {sys.executable} -m pip install azureml-train-automl-runtime==1.35.1

     |████████████████████████████████| 267 kB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 57.2 MB/s eta 0:00:01
     |████████████████████████████████| 221 kB 61.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 47.4 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 135 kB 66.8 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 705 kB/s  eta 0:00:01
     |████████████████████████████████| 39.4 MB 49 kB/s s eta 0:00:01
     |████████████████████████████████| 109 kB 51.3 MB/s eta 0:00:01
     |████████████████████████████████| 134 kB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.6 MB 54.4 MB/s eta 0:00:01
     |████████████████████████████████| 13.0 MB 57.5 MB/s eta 0:00:01
ERROR: responsibleai 0.10.0 has requirement dice-ml<0.8,>=0.7.1, but you'll have dice-ml 0.6.1 which is incompatible.
ERROR: raiwidgets 0.10.0 has requirement jinja2==2.11.3

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps =RandomParameterSampling({'C': uniform(0.5,1.5), 'max_iter': choice(50, 75, 100, 125, 150)})


# Specify a Policy
es_policy = BanditPolicy(slack_factor=0.1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', entry_script='train.py', compute_target=cpu_cluster, script_params={'--C': 'C', '--max_iter':'max_iter'})

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, policy=es_policy, primary_metric_name='Accuracy', max_duration_minutes=100, max_total_runs=10, primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, estimator=est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.core.experiment import Experiment

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [9]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_param = best_run.get_details()['runDefinition']['arguments']

print('Best parameters')
print(best_param)

LR_model = LogisticRegression(C=float(best_param[1]), max_iter=int(best_param[3]))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

LR_model.fit(x_train, y_train)

joblib.dump(LR_model, 'hyperdrive_LR.model')

Best parameters
['--C', '0.7983925455710772', '--max_iter', '50']


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


['hyperdrive_LR.model']

In [10]:
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core import Datastore


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

data_automl = x
data_automl['y'] = y

datastore = Datastore.get(ws, 'workspaceblobstore')
dataset = TabularDatasetFactory.register_pandas_dataframe(data_automl, target=datastore, name='udacity-project-data')
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target=cluster_name,
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    enable_onnx_compatible_models=True,
    enable_early_stopping=True,
    iterations=10)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/72b83965-4fd6-411e-9ac9-a6bb98286d5e/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [11]:
# Submit your automl run

exp_2 = Experiment(ws, 'AutoML_Project')
AutoML_run = exp_2.submit(automl_config, show_output=True)

RunDetails(AutoML_run).show()

Submitting remote run.
No run_configuration provided, running on udacityprojclust with default configuration
Running on remote compute: udacityprojclust


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML_Project,AutoML_172193e5-5d9d-4f63-b915-a0bf9d86d511,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [22]:
# Retrieve and get insights from your best automl model.

best_run_AutoML, fitted_model_AutoML = AutoML_run.get_output()

Package:azureml-automl-runtime, training version:1.35.1, current version:1.34.0
Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.1, current

ImportError: cannot import name 'TimeseriesDfDuplicatedIndexAutoTimeSeriesIDDetection'

In [16]:
print(fitted_model_AutoML)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
print(hasattr(fitted_model_AutoML, 'steps'))

False


In [21]:
type(fitted_model_AutoML)
for attribute in dir(fitted_model_AutoML):
    print(attribute, getattr(fitted_model_AutoML, attribute))

ByteSize <built-in method ByteSize of ModelProto object at 0x7f9044dc6620>
Clear <built-in method Clear of ModelProto object at 0x7f9044dc6620>
ClearExtension <built-in method ClearExtension of ModelProto object at 0x7f9044dc6620>
ClearField <built-in method ClearField of ModelProto object at 0x7f9044dc6620>
CopyFrom <built-in method CopyFrom of ModelProto object at 0x7f9044dc6620>
DESCRIPTOR <google.protobuf.pyext._message.MessageDescriptor object at 0x7f8fdcd80f28>
DiscardUnknownFields <built-in method DiscardUnknownFields of ModelProto object at 0x7f9044dc6620>


AttributeError: Extensions

In [15]:
from pprint import pprint

# Function to list the hyperparameters 

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators' : list(e[0] for e in step[1].estimators), 'weights' : step[1].weights})
            print()

            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        
        else:
            pprint(step[1].get_params())
            print()
        
print_model(fitted_model_AutoML)

AttributeError: steps

In [17]:
# Get information from guardrails.

print(AutoML_run.get_guardrails())


********************************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+--------------------------------------+--------------------------------------+--------------------------------------+
|Size of the smallest class            |Name/Label of the smallest class      |Number of samples in the training data|
+======================================+======================================+======================================+
|3692                                  |yes                                   |32950                         

In [18]:
# Get information from models.

print(AutoML_run.get_tags())


{'model_explain_run': 'best_run', '_aml_system_azureml.automlComponent': 'AutoML', 'pipeline_id': '', 'score': '', 'predicted_cost': '', 'fit_time': '', 'training_percent': '', 'iteration': '', 'run_preprocessor': '', 'run_algorithm': '', 'automl_best_child_run_id': 'AutoML_172193e5-5d9d-4f63-b915-a0bf9d86d511_9', 'model_explain_best_run_child_id': 'AutoML_172193e5-5d9d-4f63-b915-a0bf9d86d511_9'}


In [19]:
# Save the best model by AutoML

joblib.dump(fitted_model_AutoML, 'AutoML.model')

['AutoML.model']

In [ ]:
# Remove compute cluster

cpu_cluster.delete()